In [1]:
%matplotlib inline

from time import sleep
from random import sample

import gym
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

env = gym.make('CartPole-v0')

In [2]:
nactions =  2
nparams = 4
learning_rate = 0.05
gamma = .3

nepisode = 50

nbatch = 100
batch_legth = 512
batch_iter = 100


In [3]:
_input = tf.placeholder(shape=[1, nparams], dtype=tf.float32)

# Hidden layer 1
weight = tf.Variable(tf.random_uniform([nparams, nactions],0 , 0.01))
bias = tf.Variable(tf.random_uniform([1, nactions], 0, 0.01))
Q = tf.add(tf.matmul(_input, weight), bias)
Q = tf.sigmoid(Q)

Qa = tf.argmax(Q, 1)

# weight_1 = tf.Variable(tf.random_uniform([nparams, 10],0 , 0.01))
# bias_1 = tf.Variable(tf.random_uniform([1, 10], 0, 0.01))
# H1 = tf.add(tf.matmul(_input, weight_1), bias_1)
# H1 = tf.tanh(H1)

# weight_2 = tf.Variable(tf.random_uniform([10, nactions],0 , 0.01))
# bias_2 = tf.Variable(tf.random_uniform([1, nactions], 0, 0.01))
# Q = tf.add(tf.matmul(H1, weight_2), bias_2)
# Q = tf.tanh(Q)
# action = tf.argmax(Q, 1)

Qe = tf.placeholder(shape=[1, nactions], dtype=tf.float32)
error = tf.reduce_sum(tf.square(Qe - Q))
trainer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(error)

In [4]:

beta = 0.5
init = tf.global_variables_initializer()
steps = list()
with tf.Session() as sess:
    sess.run(init)
    
    for episode in range(nepisode):
        print()
        # training: Collect play experience
        memos = list()
        done = False
        while len(memos) < batch_legth*batch_iter:
            done = False
            state = env.reset()
            while not done:
                rand_val = np.random.rand(1)
                res_action, res_Q = sess.run([Qa, Q], feed_dict={_input:[state]})
                new_state, reward, done, _ = env.step(res_action[0])
                memos.append( (state, res_action, reward, new_state, done) )
        print("\tCollect Step in Experience: ", len(memos))
        
        # training: Review playing experience
        for iteration in range(batch_iter):
            batch = sample(memos, batch_legth)
            for state, res_action, reward, new_state, done in batch:
                r = reward
                if not done:
                    next_Q = sess.run(Q, feed_dict={_input:[new_state]})
                    r = r + gamma*np.max(next_Q)
                else:
                    r = -100
                res_Q = sess.run(Q, feed_dict={_input:[state]})
                res_Q[0, res_action[0]] = r
                sess.run([trainer], feed_dict={_input:[state], Qe:res_Q})
            
                
    
        # playing
        state = env.reset()
        step = 0
        done = False
        while not done:
            env.render()
            action = sess.run([Qa], feed_dict={_input:[state]})
            action = action[0][0]
            state, reward, done, _ = env.step(action)
            step += 1
#         print(step)
        steps.append(step)
        print("in Episode {} agent can live in {} step".format(episode, step))
plt.plot(steps)
    



	Collect Step in Experience:  51202


KeyboardInterrupt: 


0

1 [0] [[ 0.15053175  0.50103712]] [ 0.02629386  0.00242432  0.02310685  0.00609454] 
2 [0] [[ 1.15030277  0.50102186]] [ 0.02634235 -0.19302126  0.02322874  0.30597746] .
3 [0] [[ 2.15139151  0.5010094 ]] [ 0.02248192 -0.38846639  0.02934829  0.6058947 ] 
4 [1] [[ 0.52292681  3.1560545 ]] [ 0.0147126  -0.58398617  0.04146619  0.90767524] .
5 [1] [[ 0.5201813   4.15747261]] [ 0.00303287 -0.38944937  0.05961969  0.62830843] 
6 [1] [[ 0.51748788  5.1628499 ]] [-0.00475611 -0.19520793  0.07218586  0.35498176] .
7 [1] [[ 0.51483285  6.1671195 ]] [-0.00866027 -0.00118262  0.07928549  0.08590595] 
8 [1] [[ 0.51220226  7.17242479]] [-0.00868393  0.19271859  0.08100361 -0.18074665] 
9 [0] [[ 8.18975163  0.61920339]] [-0.00482955  0.3865936   0.07738868 -0.44681657] .
10 [1] [[ 0.56678742  9.18619061]] [ 0.00290232  0.19046704  0.06845235 -0.13077768] 
11 [1] [[  0.57338583  10.19913197]] [ 0.00671166  0.38454506  0.06583679 -0.40110364] .
12 [1] [[  0.57995677  11.21905422]] [ 0.01440256  0

1 [1] [[ 0.7230562   0.27555653]] [ 0.0182067  -0.02755405 -0.02716766 -0.00412023] .
2 [1] [[ 0.77176076  1.28241253]] [ 0.01765562  0.16794678 -0.02725007 -0.30524957] 
3 [1] [[ 0.81402969  2.28756213]] [ 0.02101455  0.36344625 -0.03335506 -0.60640042] .
4 [1] [[ 0.8500604   3.29136682]] [ 0.02828348  0.55901833 -0.04548307 -0.90939993] .
5 [0] [[ 4.28821468  0.97186345]] [ 0.03946385  0.75472538 -0.06367106 -1.21602434] .
6 [1] [[ 0.86335629  5.2918272 ]] [ 0.05455835  0.56047985 -0.08799155 -0.94395309] 
7 [1] [[ 0.89375955  6.29464197]] [ 0.06576795  0.75666997 -0.10687061 -1.2629359 ] 
8 [1] [[ 0.91823274  7.29656792]] [ 0.08090135  0.95298354 -0.13212933 -1.58708641] 
9 [1] [[ 0.93766183  8.29784393]] [ 0.09996102  1.14940523 -0.16387106 -1.91788391] .
9 [0] [[-99.70283508   0.99318987]] [ 0.12294913  1.34586746 -0.20222874 -2.25659206] .

9

1 [1] [[ 0.62932038  0.27891943]] [ 0.03802148  0.02190786 -0.04888613 -0.03613466] 
2 [1] [[ 0.59324473  1.2854296 ]] [ 0.03845964  0.217

5 [0] [[ 4.27013588  0.91811043]] [-0.05324431 -0.02779685  0.02287027 -0.03404136] .
6 [1] [[ 0.67475814  5.27531147]] [-0.05380024 -0.22323917  0.02218944  0.26576873] 
7 [1] [[ 0.46447074  6.28027678]] [-0.05826503 -0.02844083  0.02750482 -0.01983368] 
8 [1] [[ 0.26636747  7.28428745]] [-0.05883384  0.16627609  0.02710814 -0.3037132 ] .
9 [1] [[ 0.13161312  8.28765965]] [-0.05550832  0.36100143  0.02103388 -0.58772509] .
10 [1] [[ 0.05910698  9.29053879]] [-0.04828829  0.5558226   0.00927938 -0.87370869] 
11 [1] [[  0.02510237  10.29296303]] [-0.03717184  0.75081716 -0.0081948  -1.16345984] 
12 [1] [[  1.02886222e-02   1.12949305e+01]] [-0.0221555   0.94604485 -0.03146399 -1.45870078] 
13 [1] [[  4.10202099e-03   1.22964525e+01]] [-0.0032346   1.14153828 -0.06063801 -1.76104468] 
14 [1] [[  1.59380666e-03   1.32975760e+01]] [ 0.01959617  1.33729178 -0.0958589  -2.07195191] .
15 [1] [[  6.03137363e-04   1.42983713e+01]] [ 0.046342    1.53324787 -0.13729794 -2.39267598] .
15 [1] [[  2

5 [1] [[ 0.035232    4.28623343]] [ 0.05427796  0.82961517 -0.07551615 -1.20443381] 
6 [1] [[ 0.01474745  5.28741837]] [ 0.07087027  1.02562762 -0.09960482 -1.51979517] .
7 [1] [[  5.99627011e-03   6.28901625e+00]] [ 0.09138282  1.22180259 -0.13000073 -1.84183456] 
7 [1] [[  2.37806025e-03  -9.97090607e+01]] [ 0.11581887  1.4180976  -0.16683742 -2.17190515] 

27

1 [1] [[ 0.52958655  0.27435619]] [ 0.0449679   0.00585984  0.01351296 -0.02483615] 
2 [1] [[ 0.33065435  1.26878047]] [ 0.04508509  0.20078542  0.01301624 -0.31322514] 
3 [0] [[ 2.27428913  0.8962639 ]] [ 0.0491008   0.39571955  0.00675174 -0.6017749 ] .
4 [1] [[ 0.34051773  3.26873112]] [ 0.05701519  0.20050381 -0.00528376 -0.30697296] .
5 [1] [[ 0.1835954   4.26288509]] [ 0.06102527  0.39570065 -0.01142322 -0.60131756] .
6 [1] [[ 0.08866836  5.25743055]] [ 0.06893928  0.59098052 -0.02344957 -0.89757661] .
7 [1] [[ 0.03999167  6.25425625]] [ 0.08075889  0.78641236 -0.0414011  -1.19753731] 
8 [1] [[ 0.01728312  7.25585461]] [

16 [0] [[ 15.29743671   0.97725493]] [ 0.04213912  0.1554727  -0.08785901 -0.47097443] .
17 [0] [[ 16.29904175   0.99145705]] [ 0.04524857 -0.03830552 -0.0972785  -0.20722557] .
18 [1] [[  0.98660374  17.2969799 ]] [ 0.04448246 -0.23191157 -0.10142301  0.05325571] .
19 [0] [[ 18.29887199   0.98998719]] [ 0.03984423 -0.03549263 -0.1003579  -0.2696259 ] .
20 [1] [[  0.98693699  19.29643631]] [ 0.03913438 -0.22904989 -0.10575042 -0.01020659] .
21 [1] [[  0.98657495  20.29585838]] [ 0.03455338 -0.03258278 -0.10595455 -0.33429365] 
22 [0] [[ 21.29597664   0.96418422]] [ 0.03390172  0.16387506 -0.11264042 -0.65842022] .
23 [0] [[ 22.29838753   0.98608702]] [ 0.03717922 -0.02951385 -0.12580883 -0.40322147] 
24 [1] [[  0.98771811  23.2962532 ]] [ 0.03658895 -0.22264791 -0.13387325 -0.15270034] 
25 [0] [[ 24.29799843   0.98296714]] [ 0.03213599 -0.0258884  -0.13692726 -0.48444008] 
26 [1] [[  0.9881655   25.29640579]] [ 0.03161822 -0.21883926 -0.14661606 -0.23785372] 
27 [1] [[  0.98801887  26.

...

18 [1] [[  0.99900323  17.29978371]] [ 0.04436858 -0.19040113 -0.08250115 -0.01073394] 
19 [0] [[ 18.29984283   0.99876571]] [ 0.04056056  0.00580114 -0.08271582 -0.32826417] 
20 [1] [[  0.99905759  19.29979706]] [ 0.04067658 -0.1880518  -0.08928111 -0.06276966] 
21 [0] [[ 20.29982376   0.99861419]] [ 0.03691555  0.00822925 -0.0905365  -0.38223238] 
22 [1] [[  0.99911386  21.2998085 ]] [ 0.03708013 -0.18549837 -0.09818115 -0.11941249] 
23 [0] [[ 22.29979706   0.99842632]] [ 0.03337016  0.01088318 -0.1005694  -0.44138508] 
24 [1] [[  0.99917233  23.29982185]] [ 0.03358783 -0.18268248 -0.1093971  -0.18202127] 
25 [0] [[ 24.29977036   0.99818724]] [ 0.02993418  0.01382098 -0.11303753 -0.50711307] 
26 [0] [[ 25.29989815   0.99921465]] [ 0.0302106  -0.17954191 -0.12317979 -0.25208209] 
27 [1] [[  0.99901104  26.29978943]] [ 0.02661976 -0.37270933 -0.12822143 -0.00064975] 
28 [0] [[ 27.29987907   0.99907243]] [ 0.01916557 -0.17600393 -0.12823442 -0.33087959] 
29 [0] [[ 28.29994583   0.999594

9 [1] [[ 0.99948716  8.29988766]] [-0.01296737  0.41913518 -0.01353431 -0.6186596 ] .
10 [0] [[ 9.29984951  0.98979878]] [-0.00458467  0.61444354 -0.0259075  -0.91557427] 
11 [0] [[ 10.29979992   0.99579608]] [ 0.0077042   0.41968135 -0.04421898 -0.63114498] .
12 [0] [[ 11.2997818    0.99825591]] [ 0.01609783  0.22520336 -0.05684188 -0.3527093 ] 
13 [1] [[  0.99911481  12.2998085 ]] [ 0.0206019   0.03093384 -0.06389607 -0.07847816] 
14 [0] [[ 13.29976177   0.99810445]] [ 0.02122057  0.22691081 -0.06546563 -0.3906164 ] 
15 [1] [[  0.99915457  14.29981709]] [ 0.02575879  0.03277611 -0.07327796 -0.11927128] 
16 [0] [[ 15.29975891   0.99791485]] [ 0.02641431  0.22886729 -0.07566339 -0.43414315] 
17 [0] [[ 16.29988861   0.99911803]] [ 0.03099166  0.03489353 -0.08434625 -0.16623879] 
18 [1] [[  0.99894625  17.29977226]] [ 0.03168953 -0.15892612 -0.08767103  0.09868872] .
19 [0] [[ 18.29987335   0.99901128]] [ 0.02851101  0.0373358  -0.08569725 -0.22031578] 
20 [1] [[  0.99900669  19.29978561

8 [0] [[ 7.29984331  0.99869955]] [-0.01038877  0.14503045 -0.00638792 -0.30867012] 
9 [1] [[ 0.99910933  8.29980469]] [-0.00748816 -0.0499999  -0.01256132 -0.01800859] 
10 [0] [[ 9.29983997  0.99868053]] [-0.00848816  0.14529992 -0.0129215  -0.31462815] 
11 [0] [[ 10.29993534   0.99946707]] [-0.00558216 -0.04963561 -0.01921406 -0.02604808] .
12 [1] [[  0.99880743  11.29983616]] [-0.00657487 -0.24447682 -0.01973502  0.2605112 ] 
13 [1] [[  0.99912995  12.29980946]] [-0.01146441 -0.04907879 -0.0145248  -0.03833049] 
14 [0] [[ 13.29983234   0.99862218]] [-0.01244599  0.14624841 -0.01529141 -0.33556052] .
15 [1] [[  0.99914086  14.29981232]] [-0.00952102 -0.04865262 -0.02200262 -0.04773865] 
16 [0] [[ 15.29982758   0.9985851 ]] [-0.01049407  0.1467778  -0.02295739 -0.34728158] 
17 [1] [[  0.99915624  16.29981613]] [-0.00755851 -0.04801021 -0.02990302 -0.06192543] 
18 [0] [[ 17.29981995   0.9985317 ]] [-0.00851872  0.14752744 -0.03114153 -0.36389095] 
19 [0] [[ 18.29992676   0.99939919]] [

21 [0] [[ 20.2998867    0.99910742]] [ 0.01202861 -0.00361708 -0.07315564 -0.2551043 ] 
22 [1] [[  0.99914312  21.29981804]] [ 0.01195627 -0.19762242 -0.07825773  0.01363677] 
23 [0] [[ 22.29987526   0.9990133 ]] [ 0.00800382 -0.00147045 -0.07798499 -0.30267497] 
24 [1] [[  0.99918979  23.29982758]] [ 0.00797441 -0.19539935 -0.08403849 -0.03557009] 
25 [0] [[ 24.29986      0.99890149]] [ 0.00406642  0.00082091 -0.0847499  -0.35353998] 
26 [1] [[  0.99923766  25.29983902]] [ 0.00408284 -0.19300012 -0.0918207  -0.08873856] 
27 [0] [[ 26.29984093   0.99876475]] [  2.22838294e-04   3.30981658e-03  -9.35954663e-02  -4.08920378e-01] 
28 [1] [[  0.99928731  27.29984856]] [ 0.00028903 -0.19036913 -0.10177387 -0.14715043] 
29 [0] [[ 28.29981804   0.99859339]] [-0.00351835  0.00605185 -0.10471688 -0.47012722] 
30 [1] [[  0.99933881  29.29986   ]] [-0.00339731 -0.18744711 -0.11411943 -0.21219923] 
31 [0] [[ 30.29981804   0.99837315]] [-0.00714625  0.00910582 -0.11836341 -0.53858922] 
32 [0] [[ 31

75 [0] [[ 74.29986572   0.99892634]] [ 0.21233167  0.05697883 -0.09096066 -0.31526456] 
76 [1] [[  0.99929273  75.29985046]] [ 0.21347125 -0.13673765 -0.09726595 -0.05259667] 
77 [0] [[ 76.29984283   0.99878627]] [ 0.21073649  0.05963469 -0.09831789 -0.37431403] 
78 [1] [[  0.99934369  77.29986572]] [ 0.21192919 -0.13396324 -0.10580417 -0.11417847] 
79 [0] [[ 78.29981995   0.99861383]] [ 0.20924992  0.06250313 -0.10808774 -0.43827847] 
80 [1] [[  0.99939513  79.29987335]] [ 0.21049999 -0.13093623 -0.11685331 -0.1815287 ] 
81 [1] [[  0.9995808  80.2999115]] [ 0.20788126  0.06564709 -0.12048388 -0.50866949] .
82 [0] [[ 81.29988861   0.99575788]] [ 0.2091942   0.26224213 -0.13065727 -0.83676088] 
83 [0] [[ 82.29985046   0.99812144]] [ 0.21443905  0.06912361 -0.14739249 -0.5878562 ] 
84 [0] [[ 83.29988861   0.99916029]] [ 0.21582152 -0.12366028 -0.15914961 -0.34499246] .
85 [1] [[  0.99935704  84.29986572]] [ 0.21334831 -0.3162027  -0.16604946 -0.10642145] 
86 [0] [[ 85.29985809   0.998963

In [ ]:
a = [[  8.58846959e-03  -9.89045802e-05]]
np.argmax(a)